## Scraping tests

In [125]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Do this to enable importing modules
src_path = os.path.join(os.path.abspath(""), "..")
sys.path.insert(0, src_path)

from src.crawlers.coin_telegraph import CoinTelegraphCrawler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from selenium import webdriver
from tempfile import mkdtemp
options = webdriver.ChromeOptions()
options.binary_location = "/opt/chrome/chrome"
options.add_argument("--no-sandbox")
options.add_argument("--headless=new")
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--log-level=3")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-notifications")
options.add_argument("--disable-dev-tools")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--no-zygote")
options.add_argument(f"--user-data-dir={mkdtemp()}")
options.add_argument(f"--data-path={mkdtemp()}")
options.add_argument(f"--disk-cache-dir={mkdtemp()}")
options.add_argument("--remote-debugging-port=9222")

scroll_limit = 5

In [15]:
driver = webdriver.Chrome()

In [16]:
driver.get("https://cointelegraph.com/")

In [ ]:
# crawler = CoinTelegraphCrawler()
# crawler.extract("")

In [112]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://cointelegraph.com/")

# Create an ActionChains object
actions = ActionChains(driver)

In [113]:
main_carousel = driver.find_element(By.XPATH, "//div[@data-testid='carousel-main']")
first_li = main_carousel.find_element(By.XPATH, "//a[@data-testid='main-news-controls__link']")
first_li.click()

In [111]:
first_li.is_displayed()

True

In [100]:
from bs4 import BeautifulSoup
import time
scrolls = 0
max_scrolls = 20
last_height = 0

while True:
    # # Scroll down the page a few times
    driver.execute_script("window.scrollBy(0, 2 * window.innerHeight);")

    # Wait to load page
    time.sleep(2)

    # Break the loop if no more new content
    # new_height = driver.execute_script("return document.body.scrollHeight")
    # if new_height == last_height:
    #     break
    # last_height = new_height

    scrolls += 1
    if scrolls == max_scrolls:
        break

In [120]:
articles = []

# Parse the page source
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find all the 'article' elements
article_elements = soup.find_all('article')

# For each 'article', find the 'h1' element and all 'div' siblings
for article in article_elements:
    h1_element = article.find('h1')
    title = h1_element.text
    summary = h1_element.find_next_sibling('div').text
    content_div = article.find('div', class_='post__content-wrapper')
    time_element = article.find('time')
    datetime = time_element.get('datetime') if time_element else None
    ps = content_div.find_all('p')
    content = ' '.join(p.text for p in ps if "Advertisement" not in p.text)

    # Create a dictionary for each article and append it to the list
    articles.append({'title': title, 'summary': summary, 'content': content, 'datetime': datetime})

# Close the driver
# driver.quit()

In [121]:
import textwrap
print(textwrap.fill(articles[7]['title'], width=100))

 Weekend Wrap: SOL set for bronze, Bitcoiner fat fingers $100K fee and more


In [122]:
len(articles)

12

## Writing and reading test


In [51]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Do this to enable importing modules
src_path = os.path.join(os.path.abspath(""), "..")
sys.path.insert(0, src_path)
from src.models.documents import CoinTelegraphArticle
from src.crawlers.coin_telegraph import CoinTelegraphCrawler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
src_path

'/Users/camiloramirezf/Documents/mai-repos/text-fetch-etl/notebooks/..'

### Write to local mongo replica

Use a test db and a test collection to write


In [50]:
from pymongo import MongoClient
mongodb_uri = "mongodb://localhost:30001,localhost:30002,localhost:30003/?replicaSet=my-replica-set"
database_name = "db_test"
collection_name = "test"
test_data = {"name": "Copy cat"}

client = MongoClient(mongodb_uri)
db = client[database_name]
collection = db[collection_name]

try:
    result = collection.insert_one(test_data)
    print(f"Data inserted with _id: {result.inserted_id}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    client.close()

KeyboardInterrupt: 

### Write using crawler and the model classes

In [ ]:
crawler = CoinTelegraphCrawler()
crawler.extract()

In [52]:
# test saving articles
doc = CoinTelegraphArticle(
    title="Moon will show",
    summary="It is happening, everything is mooning",
    content="Moon is the only destiny now",
    date="2024-05-05"
)

# articles = [doc]
# crawler.save(articles)

In [54]:
doc.model_dump(exclude="id")

{'published_at': '2024-05-08',
 'title': 'Moon will show',
 'content': 'Moon is the only destiny now',
 'summary': 'It is happening, everything is mooning',
 'date': '2024-05-05'}

### Reading 


In [44]:
from pymongo import MongoClient

mongodb_uri = "mongodb://localhost:30001,localhost:30002,localhost:30003/?replicaSet=my-replica-set"
database_name = "crypto-articles"
collection_name = "articles"

client = MongoClient(mongodb_uri)
db = client[database_name]
collection = db[collection_name]

try:
    documents = collection.find({})
    for doc in documents:
        print(doc)
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    client.close()

{'_id': '872ef4a5-399f-45e3-b38a-7fd7557670b5', 'published_at': '2024-05-08', 'title': 'Moon soon', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': '61f69029-ba68-436e-936c-c5287e67b8a0', 'published_at': '2024-05-08', 'title': 'Moon later', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': 'b077e14e-0261-4d26-9f05-ce5e8e4c56d1', 'published_at': '2024-05-08', 'title': 'Moon now', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': 'cfe359f5-028f-45f3-bff7-6e30a6cec9be', 'published_at': '2024-05-08', 'title': 'Moon now', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': '4ceb0d85-35cd-4574-8afa-7a8cd2d038d9', 'published_at': '2024-05-08', 'title': 'Moon now', 'content': 'Moon is the only d